In [1]:
import ifcopenshell
import ifcopenshell.api
import ifcopenshell.guid
import ifcopenshell.util.element
import json

def create_full_4d_links():
    """
    Creates tasks, adds schedule data, creates and assigns formal resource objects,
    sequences tasks, and assigns them to a building element.
    """
    # --- Configuration ---
    input_ifc_path = "../Models/Duplex_House_Modified.ifc"
    output_ifc_path = "../Models/Duplex_House_4D_Complete.ifc"
    building_element_guid = "2FUQJjpVj9wBkoUxY59XRk"

    # --- 1. Load the IFC File and Target Element ---
    try:
        ifc_file = ifcopenshell.open(input_ifc_path)
        owner_history = ifc_file.by_type("IfcOwnerHistory")[0]
        building_element = ifc_file.by_guid(building_element_guid)
    except Exception as e:
        print(f"Error loading file or finding element: {e}")
        return

    if not building_element:
        print(f"Error: Element with GUID {building_element_guid} not found.")
        return

    print(f"Found target element: {building_element.Name}")

    # --- 2. Define Resource Data ---
    # We define the resources needed for each task.
    resource_data = {
        "Task A - Lay Foundation": [
            {"Name": "Foundation Concrete", "Type": "IfcConstructionMaterialResource"},
            {"Name": "Lead Carpenter", "Type": "IfcLaborResource"},
            {"Name": "General Helper", "Type": "IfcLaborResource"}
        ],
        "Task B - Erect Walls": [
            {"Name": "Wall Steel Frame", "Type": "IfcConstructionMaterialResource"},
            {"Name": "Lead Carpenter", "Type": "IfcLaborResource"}, # Re-using the same labor type
            {"Name": "Crane", "Type": "IfcConstructionEquipmentResource"}
        ]
    }
    
    # --- 3. Create the Tasks ---
    task_a = ifc_file.create_entity("IfcTask", GlobalId=ifcopenshell.guid.new(), Name="Task A - Lay Foundation")
    task_b = ifc_file.create_entity("IfcTask", GlobalId=ifcopenshell.guid.new(), Name="Task B - Erect Walls")
    print(f"Created Task A (ID: {task_a.id()}) and Task B (ID: {task_b.id()})")

    # --- 4. Create and Assign IfcTaskTime for each task ---
    task_time_a = ifc_file.create_entity("IfcTaskTime", ScheduleStart="2025-05-12T08:00:00", ScheduleFinish="2025-05-15T17:00:00", ScheduleDuration="P3D", ActualStart="2025-05-12T09:00:00", ActualFinish="2025-05-16T17:00:00", ActualDuration="P4D")
    task_a.TaskTime = task_time_a
    print(f"Added detailed IfcTaskTime to '{task_a.Name}'.")
    task_time_b = ifc_file.create_entity("IfcTaskTime", ScheduleStart="2025-05-16T08:00:00", ScheduleFinish="2025-05-20T17:00:00", ScheduleDuration="P4D", ActualStart="2025-05-17T08:00:00", ActualFinish="2025-05-21T17:00:00", ActualDuration="P4D")
    task_b.TaskTime = task_time_b
    print(f"Added detailed IfcTaskTime to '{task_b.Name}'.")

    # --- 5. Create Resource Objects and Assign to Tasks ---
    for task in [task_a, task_b]:
        resources_for_task = resource_data.get(task.Name, [])
        if not resources_for_task:
            continue
            
        created_resources = []
        for res_info in resources_for_task:
            resource = ifc_file.create_entity(res_info["Type"], GlobalId=ifcopenshell.guid.new(), Name=res_info["Name"])
            created_resources.append(resource)
        
        # Create the relationship to assign all created resources to the current task
        ifc_file.create_entity(
            "IfcRelAssignsToProcess",
            GlobalId=ifcopenshell.guid.new(),
            OwnerHistory=owner_history,
            RelatedObjects=created_resources,
            RelatingProcess=task
        )
        print(f"Created and assigned {len(created_resources)} resources to '{task.Name}'.")

    # --- 6. Connect the Tasks to the IfcObject (the Beam) ---
    ifc_file.create_entity("IfcRelAssignsToProduct", GlobalId=ifcopenshell.guid.new(), OwnerHistory=owner_history, RelatedObjects=[task_a], RelatingProduct=building_element)
    ifc_file.create_entity("IfcRelAssignsToProduct", GlobalId=ifcopenshell.guid.new(), OwnerHistory=owner_history, RelatedObjects=[task_b], RelatingProduct=building_element)
    print(f"Assigned both tasks to '{building_element.Name}'.")

    # --- 7. Connect the Tasks to Each Other in a Sequence ---
    ifc_file.create_entity("IfcRelSequence", GlobalId=ifcopenshell.guid.new(), OwnerHistory=owner_history, RelatingProcess=task_a, RelatedProcess=task_b, SequenceType="FINISH_START")
    print("Created Finish-to-Start sequence between Task A and Task B.")

    # --- 8. Save the Final Model ---
    ifc_file.write(output_ifc_path)
    print(f"\nSuccessfully created new 4D model at: {output_ifc_path}")


def verify_4d_links():
    """
    Loads the output file and verifies all connections, including resources.
    """
    output_ifc_path = "../Models/Duplex_House_4D_Complete.ifc"
    building_element_guid = "2FUQJjpVj9wBkoUxY59XRk"
    
    print("\n" + "="*50 + "\n--- VERIFICATION ---")
    
    try:
        ifc_file = ifcopenshell.open(output_ifc_path)
        building_element = ifc_file.by_guid(building_element_guid)
    except Exception as e:
        print(f"Verification failed. Could not open or read file '{output_ifc_path}': {e}")
        return

    # 1. Verify tasks assigned to the building element
    print(f"\n1. Verifying tasks assigned to '{building_element.Name}':")
    all_assignments = ifc_file.by_type("IfcRelAssignsToProduct")
    assigned_tasks = [obj for rel in all_assignments if rel.RelatingProduct == building_element for obj in rel.RelatedObjects if obj.is_a("IfcTask")]
    if assigned_tasks:
        for task in assigned_tasks:
            print(f"  - ✅ Found assigned task: '{task.Name}'")
    else:
        print("  - ❌ No tasks found assigned to this element.")

    # 2. Verify the sequence between the tasks
    print("\n2. Verifying sequence relationships:")
    task_a = next((t for t in ifc_file.by_type("IfcTask") if t.Name == "Task A - Lay Foundation"), None)
    if task_a and hasattr(task_a, "IsPredecessorTo") and task_a.IsPredecessorTo:
        successor = task_a.IsPredecessorTo[0].RelatedProcess
        print(f"  - ✅ Verified that '{task_a.Name}' is a predecessor to '{successor.Name}'.")
    else:
        print("  - ❌ Sequence relationship not found.")

    # 3. Verify the IfcTaskTime information for each task
    print("\n3. Verifying schedule data (IfcTaskTime):")
    all_tasks = ifc_file.by_type("IfcTask")
    for task in all_tasks:
        if task.TaskTime:
            print(f"  - ✅ Task '{task.Name}' has schedule data:")
            print(f"    - Schedule Start:  {task.TaskTime.ScheduleStart}")
            print(f"    - Schedule Finish: {task.TaskTime.ScheduleFinish}")
        else:
            print(f"  - ❌ Task '{task.Name}' is missing schedule data.")
            
    # 4. Verify the Resources assigned to each task
    print("\n4. Verifying assigned resources:")
    for task in all_tasks:
        # The inverse attribute 'IsAssignedTo' links a task to its resource assignments
        if hasattr(task, "IsAssignedTo") and task.IsAssignedTo:
            print(f"  - ✅ Task '{task.Name}' has assigned resources:")
            for rel in task.IsAssignedTo:
                if rel.is_a("IfcRelAssignsToProcess"):
                    for resource in rel.RelatedObjects:
                        print(f"    - Resource: '{resource.Name}' (Type: {resource.is_a()})")
        else:
            print(f"  - ❌ Task '{task.Name}' has no assigned resources.")


# --- Run the full process ---
create_full_4d_links()

# --- Run the verification on the file we just created ---
verify_4d_links()


Found target element: Concrete-Rectangular Beam:PBEAM_230X300:367477
Created Task A (ID: 22564) and Task B (ID: 22565)
Added detailed IfcTaskTime to 'Task A - Lay Foundation'.
Added detailed IfcTaskTime to 'Task B - Erect Walls'.
Created and assigned 3 resources to 'Task A - Lay Foundation'.
Created and assigned 3 resources to 'Task B - Erect Walls'.
Assigned both tasks to 'Concrete-Rectangular Beam:PBEAM_230X300:367477'.
Created Finish-to-Start sequence between Task A and Task B.

Successfully created new 4D model at: ../Models/Duplex_House_4D_Complete.ifc

--- VERIFICATION ---

1. Verifying tasks assigned to 'Concrete-Rectangular Beam:PBEAM_230X300:367477':
  - ✅ Found assigned task: 'Task A - Lay Foundation'
  - ✅ Found assigned task: 'Task B - Erect Walls'

2. Verifying sequence relationships:
  - ✅ Verified that 'Task A - Lay Foundation' is a predecessor to 'Task B - Erect Walls'.

3. Verifying schedule data (IfcTaskTime):
  - ✅ Task 'Task A - Lay Foundation' has schedule data:
  

In [2]:
import ifcopenshell

# --- Configuration ---
# IMPORTANT: Use the file path where you saved the model WITH the sequence relationship.
# Let's assume you saved it here:
output_ifc_path = "../Models/Duplex_House_With_Sequence.ifc"

try:
    ifc_file = ifcopenshell.open(output_ifc_path)
except OSError:
    print(f"Error: Could not find the file at '{output_ifc_path}'. Please make sure you have saved the file from the previous step.")
    exit()

# =================================================================
# Part 1: How to VERIFY the specific link you just created
# =================================================================
print("--- Part 1: Verification of the Link ---")
# Find our starting task by its name
task_a = next((task for task in ifc_file.by_type("IfcTask") if task.Name == "Task A - Lay Foundation"), None)

if task_a:
    # 'IsPredecessorTo' is the inverse attribute that finds all sequences where this task is first.
    if hasattr(task_a, "IsPredecessorTo") and task_a.IsPredecessorTo:
        # Loop through all the sequence relationships this task starts
        for rel in task_a.IsPredecessorTo:
            # The successor task is the 'RelatedProcess' in the relationship
            successor_task = rel.RelatedProcess
            dependency_type = rel.SequenceType
            print(f"✅ VERIFIED: Task '{task_a.Name}' is a predecessor to task '{successor_task.Name}'.")
            print(f"   - Dependency Type: {dependency_type}")
    else:
        print(f"Task '{task_a.Name}' was found, but it has no successors.")
else:
    print("Could not find 'Task A - Lay Foundation' in the model.")

print("\n" + "="*50 + "\n")


# =================================================================
# Part 2: General methods for QUERYING schedule information
# =================================================================
print("--- Part 2: General Querying Examples ---")

# --- Query 1: Get all tasks in the model ---
all_tasks = ifc_file.by_type("IfcTask")
print(f"\nFound {len(all_tasks)} tasks in the model:")
for task in all_tasks:
    print(f"  - Task ID: {task.id()}, Name: {task.Name}")

# --- Query 2: Get all sequence relationships in the model ---
all_sequences = ifc_file.by_type("IfcRelSequence")
print(f"\nFound {len(all_sequences)} sequence relationships:")
for seq in all_sequences:
    print(f"  - Sequence from '{seq.RelatingProcess.Name}' to '{seq.RelatedProcess.Name}'")


# --- Query 3 & 4: Create reusable functions to find predecessors and successors ---
def get_successors(task):
    """Returns a list of tasks that come after the given task."""
    successors = []
    if hasattr(task, "IsPredecessorTo") and task.IsPredecessorTo:
        for rel in task.IsPredecessorTo:
            successors.append(rel.RelatedProcess)
    return successors

def get_predecessors(task):
    """Returns a list of tasks that come before the given task."""
    predecessors = []
    if hasattr(task, "IsSuccessorFrom") and task.IsSuccessorFrom:
        for rel in task.IsSuccessorFrom:
            predecessors.append(rel.RelatingProcess)
    return predecessors

# --- Query 5: Use the functions to map out the entire schedule ---
print("\n--- Full Schedule Map ---")
for task in all_tasks:
    successors = get_successors(task)
    predecessors = get_predecessors(task)
    
    print(f"\nAnalysis for Task: '{task.Name}'")
    
    if predecessors:
        # The list comprehension '[p.Name for p in predecessors]' makes the output clean
        print(f"  - Predecessors: {[p.Name for p in predecessors]}")
    else:
        print("  - Predecessors: None (This is a starting task)")
        
    if successors:
        print(f"  - Successors:   {[s.Name for s in successors]}")
    else:
        print("  - Successors:   None (This is an ending task)")

--- Part 1: Verification of the Link ---
✅ VERIFIED: Task 'Task A - Lay Foundation' is a predecessor to task 'Task B - Erect Walls'.
   - Dependency Type: FINISH_START


--- Part 2: General Querying Examples ---

Found 2 tasks in the model:
  - Task ID: 22564, Name: Task A - Lay Foundation
  - Task ID: 22565, Name: Task B - Erect Walls

Found 1 sequence relationships:
  - Sequence from 'Task A - Lay Foundation' to 'Task B - Erect Walls'

--- Full Schedule Map ---

Analysis for Task: 'Task A - Lay Foundation'
  - Predecessors: None (This is a starting task)
  - Successors:   ['Task B - Erect Walls']

Analysis for Task: 'Task B - Erect Walls'
  - Predecessors: ['Task A - Lay Foundation']
  - Successors:   None (This is an ending task)


In [3]:
import ifcopenshell
import ifcopenshell.api
import ifcopenshell.guid
import ifcopenshell.util.element
import json

def create_full_4d_links():
    """
    Creates tasks, adds detailed schedule data (planned/actual), sequences them, 
    and assigns them to a building element.
    """
    # --- Configuration ---
    input_ifc_path = "../Models/Duplex_House_Modified.ifc"
    output_ifc_path = "../Models/Duplex_House_4D_Complete.ifc"
    building_element_guid = "2FUQJjpVj9wBkoUxY59XRk"

    # --- 1. Load the IFC File and Target Element ---
    try:
        ifc_file = ifcopenshell.open(input_ifc_path)
        owner_history = ifc_file.by_type("IfcOwnerHistory")[0]
        building_element = ifc_file.by_guid(building_element_guid)
    except Exception as e:
        print(f"Error loading file or finding element: {e}")
        return

    if not building_element:
        print(f"Error: Element with GUID {building_element_guid} not found.")
        return

    print(f"Found target element: {building_element.Name}")

    # --- 2. Create the Tasks ---
    task_a = ifc_file.create_entity("IfcTask", GlobalId=ifcopenshell.guid.new(), Name="Task A - Lay Foundation")
    task_b = ifc_file.create_entity("IfcTask", GlobalId=ifcopenshell.guid.new(), Name="Task B - Erect Walls")
    print(f"Created Task A (ID: {task_a.id()}) and Task B (ID: {task_b.id()})")

    # --- 3. Create and Assign IfcTaskTime for each task ---
    # This section now includes planned, actual, and duration values.
    
    # Create time data for Task A
    task_time_a = ifc_file.create_entity(
        "IfcTaskTime", 
        ScheduleStart="2025-05-12T08:00:00", 
        ScheduleFinish="2025-05-15T17:00:00",
        ScheduleDuration="P3D", # Planned Duration: 3 Days
        ActualStart="2025-05-12T09:00:00",
        ActualFinish="2025-05-16T17:00:00",
        ActualDuration="P4D" # Actual Duration: 4 Days
    )
    task_a.TaskTime = task_time_a
    print(f"Added detailed IfcTaskTime to '{task_a.Name}'.")

    # Create time data for Task B
    task_time_b = ifc_file.create_entity(
        "IfcTaskTime", 
        ScheduleStart="2025-05-16T08:00:00", 
        ScheduleFinish="2025-05-20T17:00:00",
        ScheduleDuration="P4D", # Planned Duration: 4 Days
        ActualStart="2025-05-17T08:00:00",
        ActualFinish="2025-05-21T17:00:00",
        ActualDuration="P4D" # Actual Duration: 4 Days
    )
    task_b.TaskTime = task_time_b
    print(f"Added detailed IfcTaskTime to '{task_b.Name}'.")

    # --- 4. Connect the Tasks to the IfcObject (the Beam) ---
    ifc_file.create_entity(
        "IfcRelAssignsToProduct", GlobalId=ifcopenshell.guid.new(), OwnerHistory=owner_history,
        RelatedObjects=[task_a], RelatingProduct=building_element
    )
    ifc_file.create_entity(
        "IfcRelAssignsToProduct", GlobalId=ifcopenshell.guid.new(), OwnerHistory=owner_history,
        RelatedObjects=[task_b], RelatingProduct=building_element
    )
    print(f"Assigned both tasks to '{building_element.Name}'.")

    # --- 5. Connect the Tasks to Each Other in a Sequence ---
    ifc_file.create_entity(
        "IfcRelSequence", GlobalId=ifcopenshell.guid.new(), OwnerHistory=owner_history,
        RelatingProcess=task_a, RelatedProcess=task_b, SequenceType="FINISH_START"
    )
    print("Created Finish-to-Start sequence between Task A and Task B.")

    # --- 6. Save the Final Model ---
    ifc_file.write(output_ifc_path)
    print(f"\nSuccessfully created new 4D model at: {output_ifc_path}")


def verify_4d_links():
    """
    Loads the output file and verifies all connections, including detailed TaskTime.
    """
    output_ifc_path = "../Models/Duplex_House_4D_Complete.ifc"
    building_element_guid = "2FUQJjpVj9wBkoUxY59XRk"
    
    print("\n" + "="*50 + "\n--- VERIFICATION ---")
    
    try:
        ifc_file = ifcopenshell.open(output_ifc_path)
        building_element = ifc_file.by_guid(building_element_guid)
    except Exception as e:
        print(f"Verification failed. Could not open or read file '{output_ifc_path}': {e}")
        return

    # 1. Verify tasks assigned to the building element
    print(f"\n1. Verifying tasks assigned to '{building_element.Name}':")
    all_assignments = ifc_file.by_type("IfcRelAssignsToProduct")
    assigned_tasks = []
    for rel in all_assignments:
        if rel.RelatingProduct == building_element:
            for obj in rel.RelatedObjects:
                if obj.is_a("IfcTask"):
                    assigned_tasks.append(obj)
    
    if assigned_tasks:
        for task in assigned_tasks:
            print(f"  - ✅ Found assigned task: '{task.Name}'")
    else:
        print("  - ❌ No tasks found assigned to this element.")

    # 2. Verify the sequence between the tasks
    print("\n2. Verifying sequence relationships:")
    task_a = next((t for t in ifc_file.by_type("IfcTask") if t.Name == "Task A - Lay Foundation"), None)
    if task_a and hasattr(task_a, "IsPredecessorTo") and task_a.IsPredecessorTo:
        successor = task_a.IsPredecessorTo[0].RelatedProcess
        print(f"  - ✅ Verified that '{task_a.Name}' is a predecessor to '{successor.Name}'.")
    else:
        print("  - ❌ Sequence relationship not found.")

    # 3. Verify the IfcTaskTime information for each task
    print("\n3. Verifying schedule data (IfcTaskTime):")
    all_tasks = ifc_file.by_type("IfcTask")
    for task in all_tasks:
        if task.TaskTime:
            print(f"  - ✅ Task '{task.Name}' has schedule data:")
            print(f"    - Schedule Start:  {task.TaskTime.ScheduleStart}")
            print(f"    - Schedule Finish: {task.TaskTime.ScheduleFinish}")
            print(f"    - Schedule Duration: {task.TaskTime.ScheduleDuration}")
            print(f"    - Actual Start:    {task.TaskTime.ActualStart}")
            print(f"    - Actual Finish:   {task.TaskTime.ActualFinish}")
            print(f"    - Actual Duration:   {task.TaskTime.ActualDuration}")
        else:
            print(f"  - ❌ Task '{task.Name}' is missing schedule data.")


# --- Run the full process ---
create_full_4d_links()

# --- Run the verification on the file we just created ---
verify_4d_links()


Found target element: Concrete-Rectangular Beam:PBEAM_230X300:367477
Created Task A (ID: 22564) and Task B (ID: 22565)
Added detailed IfcTaskTime to 'Task A - Lay Foundation'.
Added detailed IfcTaskTime to 'Task B - Erect Walls'.
Assigned both tasks to 'Concrete-Rectangular Beam:PBEAM_230X300:367477'.
Created Finish-to-Start sequence between Task A and Task B.

Successfully created new 4D model at: ../Models/Duplex_House_4D_Complete.ifc

--- VERIFICATION ---

1. Verifying tasks assigned to 'Concrete-Rectangular Beam:PBEAM_230X300:367477':
  - ✅ Found assigned task: 'Task A - Lay Foundation'
  - ✅ Found assigned task: 'Task B - Erect Walls'

2. Verifying sequence relationships:
  - ✅ Verified that 'Task A - Lay Foundation' is a predecessor to 'Task B - Erect Walls'.

3. Verifying schedule data (IfcTaskTime):
  - ✅ Task 'Task A - Lay Foundation' has schedule data:
    - Schedule Start:  2025-05-12T08:00:00
    - Schedule Finish: 2025-05-15T17:00:00
    - Schedule Duration: P3D
    - Actu

In [5]:
import ifcopenshell
import ifcopenshell.api
import ifcopenshell.guid
import ifcopenshell.util.element
import json

def create_full_4d_links():
    """
    Creates tasks, adds schedule data, creates and assigns formal resource objects,
    sequences tasks, and assigns them to a building element.
    """
    # --- Configuration ---
    input_ifc_path = "../Models/Duplex_House_Modified.ifc"
    output_ifc_path = "../Models/Duplex_House_4D_Complete.ifc"
    building_element_guid = "2FUQJjpVj9wBkoUxY59XRk"

    # --- 1. Load the IFC File and Target Element ---
    try:
        ifc_file = ifcopenshell.open(input_ifc_path)
        owner_history = ifc_file.by_type("IfcOwnerHistory")[0]
        building_element = ifc_file.by_guid(building_element_guid)
    except Exception as e:
        print(f"Error loading file or finding element: {e}")
        return

    if not building_element:
        print(f"Error: Element with GUID {building_element_guid} not found.")
        return

    print(f"Found target element: {building_element.Name}")

    # --- 2. Define Resource Data ---
    resource_data = {
        "Task A - Lay Foundation": [
            {"Name": "Foundation Concrete", "Type": "IfcConstructionMaterialResource"},
            {"Name": "Lead Carpenter", "Type": "IfcLaborResource"},
            {"Name": "General Helper", "Type": "IfcLaborResource"}
        ],
        "Task B - Erect Walls": [
            {"Name": "Wall Steel Frame", "Type": "IfcConstructionMaterialResource"},
            {"Name": "Lead Carpenter", "Type": "IfcLaborResource"}, 
            {"Name": "Crane", "Type": "IfcConstructionEquipmentResource"}
        ]
    }
    
    # --- 3. Create the Tasks ---
    task_a = ifc_file.create_entity("IfcTask", GlobalId=ifcopenshell.guid.new(), Name="Task A - Lay Foundation")
    task_b = ifc_file.create_entity("IfcTask", GlobalId=ifcopenshell.guid.new(), Name="Task B - Erect Walls")
    print(f"Created Task A (ID: {task_a.id()}) and Task B (ID: {task_b.id()})")

    # --- 4. Create and Assign IfcTaskTime for each task ---
    task_time_a = ifc_file.create_entity("IfcTaskTime", ScheduleStart="2025-05-12T08:00:00", ScheduleFinish="2025-05-15T17:00:00", ScheduleDuration="P3D", ActualStart="2025-05-12T09:00:00", ActualFinish="2025-05-16T17:00:00", ActualDuration="P4D")
    task_a.TaskTime = task_time_a
    print(f"Added detailed IfcTaskTime to '{task_a.Name}'.")
    task_time_b = ifc_file.create_entity("IfcTaskTime", ScheduleStart="2025-05-16T08:00:00", ScheduleFinish="2025-05-20T17:00:00", ScheduleDuration="P4D", ActualStart="2025-05-17T08:00:00", ActualFinish="2025-05-21T17:00:00", ActualDuration="P4D")
    task_b.TaskTime = task_time_b
    print(f"Added detailed IfcTaskTime to '{task_b.Name}'.")

    # --- 5. Create Resource Objects and Assign to Tasks ---
    for task in [task_a, task_b]:
        resources_for_task = resource_data.get(task.Name, [])
        if not resources_for_task:
            continue
            
        created_resources = []
        for res_info in resources_for_task:
            resource = ifc_file.create_entity(res_info["Type"], GlobalId=ifcopenshell.guid.new(), Name=res_info["Name"])
            created_resources.append(resource)
        
        ifc_file.create_entity(
            "IfcRelAssignsToProcess",
            GlobalId=ifcopenshell.guid.new(),
            OwnerHistory=owner_history,
            RelatedObjects=created_resources,
            RelatingProcess=task
        )
        print(f"Created and assigned {len(created_resources)} resources to '{task.Name}'.")

    # --- 6. Connect the Tasks to the IfcObject (the Beam) ---
    ifc_file.create_entity("IfcRelAssignsToProduct", GlobalId=ifcopenshell.guid.new(), OwnerHistory=owner_history, RelatedObjects=[task_a], RelatingProduct=building_element)
    ifc_file.create_entity("IfcRelAssignsToProduct", GlobalId=ifcopenshell.guid.new(), OwnerHistory=owner_history, RelatedObjects=[task_b], RelatingProduct=building_element)
    print(f"Assigned both tasks to '{building_element.Name}'.")

    # --- 7. Connect the Tasks to Each Other in a Sequence ---
    ifc_file.create_entity("IfcRelSequence", GlobalId=ifcopenshell.guid.new(), OwnerHistory=owner_history, RelatingProcess=task_a, RelatedProcess=task_b, SequenceType="FINISH_START")
    print("Created Finish-to-Start sequence between Task A and Task B.")

    # --- 8. Save the Final Model ---
    ifc_file.write(output_ifc_path)
    print(f"\nSuccessfully created new 4D model at: {output_ifc_path}")


def verify_4d_links():
    """
    Loads the output file and verifies all connections, including resources.
    """
    output_ifc_path = "../Models/Duplex_House_4D_Complete.ifc"
    building_element_guid = "2FUQJjpVj9wBkoUxY59XRk"
    
    print("\n" + "="*50 + "\n--- VERIFICATION ---")
    
    try:
        ifc_file = ifcopenshell.open(output_ifc_path)
        building_element = ifc_file.by_guid(building_element_guid)
    except Exception as e:
        print(f"Verification failed. Could not open or read file '{output_ifc_path}': {e}")
        return

    # 1. Verify tasks assigned to the building element
    print(f"\n1. Verifying tasks assigned to '{building_element.Name}':")
    all_product_assignments = ifc_file.by_type("IfcRelAssignsToProduct")
    assigned_tasks = [obj for rel in all_product_assignments if rel.RelatingProduct == building_element for obj in rel.RelatedObjects if obj.is_a("IfcTask")]
    if assigned_tasks:
        for task in assigned_tasks:
            print(f"  - ✅ Found assigned task: '{task.Name}'")
    else:
        print("  - ❌ No tasks found assigned to this element.")

    # 2. Verify the sequence between the tasks
    print("\n2. Verifying sequence relationships:")
    task_a = next((t for t in ifc_file.by_type("IfcTask") if t.Name == "Task A - Lay Foundation"), None)
    if task_a and hasattr(task_a, "IsPredecessorTo") and task_a.IsPredecessorTo:
        successor = task_a.IsPredecessorTo[0].RelatedProcess
        print(f"  - ✅ Verified that '{task_a.Name}' is a predecessor to '{successor.Name}'.")
    else:
        print("  - ❌ Sequence relationship not found.")

    # 3. Verify the IfcTaskTime information for each task
    print("\n3. Verifying schedule data (IfcTaskTime):")
    all_tasks = ifc_file.by_type("IfcTask")
    for task in all_tasks:
        if task.TaskTime:
            print(f"  - ✅ Task '{task.Name}' has schedule data:")
            print(f"    - Schedule Start:  {task.TaskTime.ScheduleStart}")
            print(f"    - Schedule Finish: {task.TaskTime.ScheduleFinish}")
        else:
            print(f"  - ❌ Task '{task.Name}' is missing schedule data.")
            
    # 4. Verify the Resources assigned to each task
    print("\n4. Verifying assigned resources:")
    all_process_assignments = ifc_file.by_type("IfcRelAssignsToProcess")
    for task in all_tasks:
        resources_for_this_task = []
        # --- CORRECTED VERIFICATION LOGIC ---
        # Query all resource assignments and check which ones point to the current task.
        for rel in all_process_assignments:
            if rel.RelatingProcess == task:
                resources_for_this_task.extend(rel.RelatedObjects)
        
        if resources_for_this_task:
            print(f"  - ✅ Task '{task.Name}' has assigned resources:")
            for resource in resources_for_this_task:
                print(f"    - Resource: '{resource.Name}' (Type: {resource.is_a()})")
        else:
            print(f"  - ❌ Task '{task.Name}' has no assigned resources.")


# --- Run the full process ---
create_full_4d_links()

# --- Run the verification on the file we just created ---
verify_4d_links()


Found target element: Concrete-Rectangular Beam:PBEAM_230X300:367477
Created Task A (ID: 22564) and Task B (ID: 22565)
Added detailed IfcTaskTime to 'Task A - Lay Foundation'.
Added detailed IfcTaskTime to 'Task B - Erect Walls'.
Created and assigned 3 resources to 'Task A - Lay Foundation'.
Created and assigned 3 resources to 'Task B - Erect Walls'.
Assigned both tasks to 'Concrete-Rectangular Beam:PBEAM_230X300:367477'.
Created Finish-to-Start sequence between Task A and Task B.

Successfully created new 4D model at: ../Models/Duplex_House_4D_Complete.ifc

--- VERIFICATION ---

1. Verifying tasks assigned to 'Concrete-Rectangular Beam:PBEAM_230X300:367477':
  - ✅ Found assigned task: 'Task A - Lay Foundation'
  - ✅ Found assigned task: 'Task B - Erect Walls'

2. Verifying sequence relationships:
  - ✅ Verified that 'Task A - Lay Foundation' is a predecessor to 'Task B - Erect Walls'.

3. Verifying schedule data (IfcTaskTime):
  - ✅ Task 'Task A - Lay Foundation' has schedule data:
  

In [7]:
import ifcopenshell
import ifcopenshell.api
import ifcopenshell.guid
import ifcopenshell.util.element
import json

def create_full_5d_links():
    """
    Creates tasks, adds schedule data, creates and assigns formal resource objects
    with quantitative data, adds cost items, sequences tasks, and assigns them to a building element.
    """
    # --- Configuration ---
    input_ifc_path = "../Models/Duplex_House_Modified.ifc"
    output_ifc_path = "../Models/Duplex_House_5D_Complete.ifc" # New output file
    building_element_guid = "2FUQJjpVj9wBkoUxY59XRk"

    # --- 1. Load the IFC File and Target Element ---
    try:
        ifc_file = ifcopenshell.open(input_ifc_path)
        owner_history = ifc_file.by_type("IfcOwnerHistory")[0]
        building_element = ifc_file.by_guid(building_element_guid)
    except Exception as e:
        print(f"Error loading file or finding element: {e}")
        return

    if not building_element:
        print(f"Error: Element with GUID {building_element_guid} not found.")
        return

    print(f"Found target element: {building_element.Name}")

    # --- 2. Define Resource and Cost Data ---
    task_data = {
        "Task A - Lay Foundation": {
            "Resources": [
                {"Name": "Foundation Concrete", "Type": "IfcConstructionMaterialResource", "BudgetedUnits": 30.0, "ActualUnits": 32.5, "UnitPrice": 150.0},
                {"Name": "Lead Carpenter", "Type": "IfcLaborResource", "BudgetedUnits": 24.0, "ActualUnits": 26.0, "UnitPrice": 50.0},
                {"Name": "General Helper", "Type": "IfcLaborResource", "BudgetedUnits": 48.0, "ActualUnits": 50.0, "UnitPrice": 25.0}
            ],
            "Costs": [
                {"Name": "Foundation Labor Cost", "CostValue": 2500.0},
                {"Name": "Foundation Material Cost", "CostValue": 4500.0}
            ]
        },
        "Task B - Erect Walls": {
            "Resources": [
                {"Name": "Wall Steel Frame", "Type": "IfcConstructionMaterialResource", "BudgetedUnits": 500.0, "ActualUnits": 512.0, "UnitPrice": 2.5},
                {"Name": "Lead Carpenter", "Type": "IfcLaborResource", "BudgetedUnits": 16.0, "ActualUnits": 16.0, "UnitPrice": 50.0},
                {"Name": "Crane", "Type": "IfcConstructionEquipmentResource", "BudgetedUnits": 8.0, "ActualUnits": 9.5, "UnitPrice": 200.0}
            ],
            "Costs": [
                 {"Name": "Walls Labor Cost", "CostValue": 1000.0},
                 {"Name": "Walls Material Cost", "CostValue": 1280.0},
                 {"Name": "Walls Equipment Cost", "CostValue": 1900.0}
            ]
        }
    }
    
    # --- 3. Create the Tasks ---
    task_a = ifc_file.create_entity("IfcTask", GlobalId=ifcopenshell.guid.new(), Name="Task A - Lay Foundation")
    task_b = ifc_file.create_entity("IfcTask", GlobalId=ifcopenshell.guid.new(), Name="Task B - Erect Walls")
    print(f"Created Task A (ID: {task_a.id()}) and Task B (ID: {task_b.id()})")

    # --- 4. Create and Assign IfcTaskTime for each task ---
    task_time_a = ifc_file.create_entity("IfcTaskTime", ScheduleStart="2025-05-12T08:00:00", ScheduleFinish="2025-05-15T17:00:00", ScheduleDuration="P3D", ActualStart="2025-05-12T09:00:00", ActualFinish="2025-05-16T17:00:00", ActualDuration="P4D")
    task_a.TaskTime = task_time_a
    task_time_b = ifc_file.create_entity("IfcTaskTime", ScheduleStart="2025-05-16T08:00:00", ScheduleFinish="2025-05-20T17:00:00", ScheduleDuration="P4D", ActualStart="2025-05-17T08:00:00", ActualFinish="2025-05-21T17:00:00", ActualDuration="P4D")
    task_b.TaskTime = task_time_b
    print("Added detailed IfcTaskTime to both tasks.")

    # --- 5. Process Tasks: Assign Resources and Costs ---
    for task in [task_a, task_b]:
        data_for_task = task_data.get(task.Name, {})
        
        # A) Create Resource Objects, Add Psets, and Assign to Task
        resources_for_task = data_for_task.get("Resources", [])
        created_resources = []
        for res_info in resources_for_task:
            resource = ifc_file.create_entity(res_info["Type"], GlobalId=ifcopenshell.guid.new(), Name=res_info["Name"])
            pset = ifcopenshell.api.run("pset.add_pset", ifc_file, product=resource, name="Pset_ResourceData")
            ifcopenshell.api.run("pset.edit_pset", ifc_file, pset=pset, properties={"BudgetedUnits": res_info["BudgetedUnits"], "ActualUnits": res_info["ActualUnits"], "UnitPrice": res_info["UnitPrice"]})
            created_resources.append(resource)
        ifc_file.create_entity("IfcRelAssignsToProcess", GlobalId=ifcopenshell.guid.new(), OwnerHistory=owner_history, RelatedObjects=created_resources, RelatingProcess=task)
        print(f"Created, added data to, and assigned {len(created_resources)} resources to '{task.Name}'.")

        # B) Create Cost Items and Assign to Task
        costs_for_task = data_for_task.get("Costs", [])
        created_cost_items = []
        for cost_info in costs_for_task:
            # CORRECTED: Create an IfcMonetaryMeasure first to wrap the float value.
            monetary_value = ifc_file.create_entity("IfcMonetaryMeasure", cost_info["CostValue"])
            
            # Now create the IfcCostValue using the IfcMonetaryMeasure instance.
            cost_value = ifc_file.create_entity("IfcCostValue", Name=cost_info["Name"], AppliedValue=monetary_value, UnitBasis=None)
            
            cost_item = ifc_file.create_entity("IfcCostItem", GlobalId=ifcopenshell.guid.new(), Name=cost_info["Name"], CostValues=[cost_value])
            created_cost_items.append(cost_item)
        ifc_file.create_entity("IfcRelAssignsToControl", GlobalId=ifcopenshell.guid.new(), OwnerHistory=owner_history, RelatedObjects=created_cost_items, RelatingControl=task)
        print(f"Created and assigned {len(created_cost_items)} cost items to '{task.Name}'.")


    # --- 6. Connect the Tasks to the IfcObject (the Beam) ---
    ifc_file.create_entity("IfcRelAssignsToProduct", GlobalId=ifcopenshell.guid.new(), OwnerHistory=owner_history, RelatedObjects=[task_a], RelatingProduct=building_element)
    ifc_file.create_entity("IfcRelAssignsToProduct", GlobalId=ifcopenshell.guid.new(), OwnerHistory=owner_history, RelatedObjects=[task_b], RelatingProduct=building_element)
    print(f"Assigned both tasks to '{building_element.Name}'.")

    # --- 7. Connect the Tasks to Each Other in a Sequence ---
    ifc_file.create_entity("IfcRelSequence", GlobalId=ifcopenshell.guid.new(), OwnerHistory=owner_history, RelatingProcess=task_a, RelatedProcess=task_b, SequenceType="FINISH_START")
    print("Created Finish-to-Start sequence between Task A and Task B.")

    # --- 8. Save the Final Model ---
    ifc_file.write(output_ifc_path)
    print(f"\nSuccessfully created new 5D model at: {output_ifc_path}")


def verify_5d_links():
    """
    Loads the output file and verifies all connections, including resource and cost properties.
    """
    output_ifc_path = "../Models/Duplex_House_5D_Complete.ifc"
    
    print("\n" + "="*50 + "\n--- VERIFICATION ---")
    
    try:
        ifc_file = ifcopenshell.open(output_ifc_path)
    except Exception as e:
        print(f"Verification failed. Could not open or read file '{output_ifc_path}': {e}")
        return

    all_tasks = ifc_file.by_type("IfcTask")
    if not all_tasks:
        print("No tasks found in the model.")
        return

    # Verify Resources and Costs for each task
    all_process_assignments = ifc_file.by_type("IfcRelAssignsToProcess")
    all_control_assignments = ifc_file.by_type("IfcRelAssignsToControl")

    for task in all_tasks:
        print(f"\n--- Verifying Task: '{task.Name}' ---")

        # Verify Resources
        resources_for_this_task = [obj for rel in all_process_assignments if rel.RelatingProcess == task for obj in rel.RelatedObjects]
        if resources_for_this_task:
            print(f"  - ✅ Has assigned resources:")
            for resource in resources_for_this_task:
                print(f"    - Resource: '{resource.Name}'")
                psets = ifcopenshell.util.element.get_psets(resource, psets_only=True)
                if "Pset_ResourceData" in psets:
                    print("      - ✅ Found Pset_ResourceData:", psets["Pset_ResourceData"])
        else:
            print(f"  - ❌ Task has no assigned resources.")

        # Verify Costs
        costs_for_this_task = [obj for rel in all_control_assignments if rel.RelatingControl == task for obj in rel.RelatedObjects]
        if costs_for_this_task:
            print(f"  - ✅ Has assigned costs:")
            for cost_item in costs_for_this_task:
                if cost_item.is_a("IfcCostItem") and cost_item.CostValues:
                    # CORRECTED: Access the wrapped float value from the IfcMonetaryMeasure
                    value_entity = cost_item.CostValues[0].AppliedValue
                    print(f"    - Cost Item: '{cost_item.Name}', Value: {value_entity.wrappedValue}")
        else:
            print(f"  - ❌ Task has no assigned costs.")


# --- Run the full process ---
create_full_5d_links()

# --- Run the verification on the file we just created ---
verify_5d_links()


Found target element: Concrete-Rectangular Beam:PBEAM_230X300:367477
Created Task A (ID: 22564) and Task B (ID: 22565)
Added detailed IfcTaskTime to both tasks.
Created, added data to, and assigned 3 resources to 'Task A - Lay Foundation'.
Created and assigned 2 cost items to 'Task A - Lay Foundation'.
Created, added data to, and assigned 3 resources to 'Task B - Erect Walls'.
Created and assigned 3 cost items to 'Task B - Erect Walls'.
Assigned both tasks to 'Concrete-Rectangular Beam:PBEAM_230X300:367477'.
Created Finish-to-Start sequence between Task A and Task B.

Successfully created new 5D model at: ../Models/Duplex_House_5D_Complete.ifc

--- VERIFICATION ---

--- Verifying Task: 'Task A - Lay Foundation' ---
  - ✅ Has assigned resources:
    - Resource: 'Foundation Concrete'
      - ✅ Found Pset_ResourceData: {'BudgetedUnits': 30.0, 'ActualUnits': 32.5, 'UnitPrice': 150.0, 'id': 22570}
    - Resource: 'Lead Carpenter'
      - ✅ Found Pset_ResourceData: {'BudgetedUnits': 24.0, 'A

In [8]:
import ifcopenshell
import ifcopenshell.api
import ifcopenshell.guid
import ifcopenshell.util.element
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from urllib.error import URLError

def run_sparql_query(building_element_guid):
    """
    Queries the ontology for all data related to a specific building element GUID.
    """
    sparql = SPARQLWrapper("http://localhost:3030/bimontoDB/sparql")
    
    # This query has been updated to match your new ontology schema.
    query = f"""
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX bimonto: <http://w3id.org/IproK/00/BIMOnto#>
        PREFIX iprok: <http://w3id.org/IproK#>

        SELECT *
        WHERE {{
          ?building_element bimonto:GlobalId "{building_element_guid}" ;
                            bimonto:AssignToProcess ?task .
          
          ?task iprok:Name ?TaskName .
          
          OPTIONAL {{
            ?task iprok:hasTaskTime ?task_time .
            ?task_time iprok:ScheduleStart ?ScheduleStart ;
                       iprok:ScheduleFinish ?ScheduleFinish ;
                       iprok:ScheduleDuration ?ScheduleDuration ;
                       iprok:ActualStart ?ActualStart ;
                       iprok:ActualFinish ?ActualFinish ;
                       iprok:ActualDuration ?ActualDuration .
          }}
          
          OPTIONAL {{
            ?task iprok:requiresResource ?resource .
            ?resource rdf:type ?ResourceType ;
                      iprok:BudgetedUnits ?BudgetedUnits ;
                      iprok:ActualUnits ?ActualUnits .
          }}
          
          OPTIONAL {{
            ?task iprok:hasCostItem ?cost_item .
            ?cost_item iprok:hasCostType ?CostType ;
                       iprok:ActualCost ?ActualCost ;
                       iprok:BudgetedCost ?BudgetedCost .
          }}
        }}
    """
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    try:
        print("Querying SPARQL endpoint...")
        results = sparql.query().convert()
        print("Query successful.")
        return results["results"]["bindings"]
    except URLError as e:
        print(f"Error connecting to SPARQL endpoint: {e}")
    except Exception as e:
        print(f"An error occurred during SPARQL query: {e}")
    return None

def process_sparql_results(bindings):
    """
    Transforms the flat list of SPARQL results into a structured, nested dictionary
    based on the new ontology schema.
    """
    if not bindings:
        return {}
        
    tasks = {}
    for row in bindings:
        task_name = row.get("TaskName", {}).get("value")
        if not task_name:
            continue

        # Initialize the task entry if it's the first time we see it
        if task_name not in tasks:
            tasks[task_name] = {
                "Time": {
                    "ScheduleStart": row.get("ScheduleStart", {}).get("value"),
                    "ScheduleFinish": row.get("ScheduleFinish", {}).get("value"),
                    "ScheduleDuration": row.get("ScheduleDuration", {}).get("value"),
                    "ActualStart": row.get("ActualStart", {}).get("value"),
                    "ActualFinish": row.get("ActualFinish", {}).get("value"),
                    "ActualDuration": row.get("ActualDuration", {}).get("value")
                },
                "Resources": {}, # Use dicts to avoid duplicates
                "Costs": {}
            }

        # Add resource data if it exists in this row
        resource_uri = row.get("resource", {}).get("value")
        resource_type_uri = row.get("ResourceType", {}).get("value", "")
        if resource_uri and resource_uri not in tasks[task_name]["Resources"] and "NamedIndividual" not in resource_type_uri:
            resource_name = resource_type_uri.split('#')[-1]
            tasks[task_name]["Resources"][resource_uri] = {
                "Name": resource_name,
                "Type": f"Ifc{resource_name}Resource", # e.g. IfcConcreteResource - assuming a mapping
                "BudgetedUnits": float(row.get("BudgetedUnits", {}).get("value", 0)),
                "ActualUnits": float(row.get("ActualUnits", {}).get("value", 0))
            }

        # Add cost data if it exists in this row
        cost_item_uri = row.get("cost_item", {}).get("value")
        cost_type_uri = row.get("CostType", {}).get("value", "")
        if cost_item_uri and cost_item_uri not in tasks[task_name]["Costs"]:
            cost_type_name = cost_type_uri.split('#')[-1]
            tasks[task_name]["Costs"][cost_item_uri] = {
                "Name": f"{cost_type_name} Cost", # e.g. "Labor Cost"
                "BudgetedCost": float(row.get("BudgetedCost", {}).get("value", 0)),
                "ActualCost": float(row.get("ActualCost", {}).get("value", 0))
            }
            
    # Convert the inner resource and cost dicts to lists
    for task_name, task_data in tasks.items():
        task_data["Resources"] = list(task_data["Resources"].values())
        task_data["Costs"] = list(task_data["Costs"].values())

    print("Successfully processed SPARQL results into a structured format.")
    return tasks


def create_5d_model_from_ontology(building_element_guid):
    """
    Main function to drive the entire process from query to IFC creation,
    updated for the new ontology schema.
    """
    # --- 1. Get Data from Ontology ---
    sparql_bindings = run_sparql_query(building_element_guid)
    if not sparql_bindings:
        print("Could not retrieve data from ontology. Aborting.")
        return
        
    task_data = process_sparql_results(sparql_bindings)

    # --- 2. Load IFC Template and Target Element ---
    input_ifc_path = "../Models/Duplex_House_Modified.ifc"
    output_ifc_path = "../Models/Duplex_House_5D_From_Ontology.ifc"
    
    try:
        ifc_file = ifcopenshell.open(input_ifc_path)
        owner_history = ifc_file.by_type("IfcOwnerHistory")[0]
        building_element = ifc_file.by_guid(building_element_guid)
    except Exception as e:
        print(f"Error loading file or finding element: {e}")
        return

    print(f"\nFound target element in IFC: {building_element.Name}")

    # --- 3. Create IFC Entities from Processed Data ---
    created_tasks = {}
    for task_name, data_for_task in task_data.items():
        # Create Task
        task = ifc_file.create_entity("IfcTask", GlobalId=ifcopenshell.guid.new(), Name=task_name)
        created_tasks[task_name] = task
        print(f"Created IfcTask: '{task_name}'")

        # Create and Assign TaskTime
        time_data = data_for_task.get("Time", {})
        task_time = ifc_file.create_entity("IfcTaskTime", **time_data)
        task.TaskTime = task_time

        # Create, Add Psets to, and Assign Resources
        resources_for_task = data_for_task.get("Resources", [])
        created_resources = []
        for res_info in resources_for_task:
            # A simple mapping from ontology type to IFC resource type
            ifc_resource_type = "IfcConstructionMaterialResource" # Default
            if "Labor" in res_info["Type"] or "Carpenter" in res_info["Type"]:
                ifc_resource_type = "IfcLaborResource"
            elif "Equipment" in res_info["Type"] or "Crane" in res_info["Type"]:
                ifc_resource_type = "IfcConstructionEquipmentResource"

            resource = ifc_file.create_entity(ifc_resource_type, GlobalId=ifcopenshell.guid.new(), Name=res_info["Name"])
            pset = ifcopenshell.api.run("pset.add_pset", ifc_file, product=resource, name="Pset_ResourceQuantities")
            ifcopenshell.api.run("pset.edit_pset", ifc_file, pset=pset, properties={"BudgetedUnits": res_info["BudgetedUnits"], "ActualUnits": res_info["ActualUnits"]})
            created_resources.append(resource)
        ifc_file.create_entity("IfcRelAssignsToProcess", GlobalId=ifcopenshell.guid.new(), OwnerHistory=owner_history, RelatedObjects=created_resources, RelatingProcess=task)

        # Create and Assign Cost Items
        costs_for_task = data_for_task.get("Costs", [])
        created_cost_items = []
        for cost_info in costs_for_task:
            # Create a CostItem for each cost type (e.g., Labor Cost)
            cost_item = ifc_file.create_entity("IfcCostItem", GlobalId=ifcopenshell.guid.new(), Name=cost_info["Name"])
            
            # Create two CostValues (Budgeted and Actual) and assign them to the CostItem
            budgeted_monetary_value = ifc_file.create_entity("IfcMonetaryMeasure", cost_info["BudgetedCost"])
            budgeted_cost_value = ifc_file.create_entity("IfcCostValue", Name="Budgeted", AppliedValue=budgeted_monetary_value)
            
            actual_monetary_value = ifc_file.create_entity("IfcMonetaryMeasure", cost_info["ActualCost"])
            actual_cost_value = ifc_file.create_entity("IfcCostValue", Name="Actual", AppliedValue=actual_monetary_value)
            
            cost_item.CostValues = [budgeted_cost_value, actual_cost_value]
            created_cost_items.append(cost_item)
            
        ifc_file.create_entity("IfcRelAssignsToControl", GlobalId=ifcopenshell.guid.new(), OwnerHistory=owner_history, RelatedObjects=created_cost_items, RelatingControl=task)
        
        # Assign Task to Building Element
        ifc_file.create_entity("IfcRelAssignsToProduct", GlobalId=ifcopenshell.guid.new(), OwnerHistory=owner_history, RelatedObjects=[task], RelatingProduct=building_element)

    # --- 4. (Optional) Create Sequences if order is known ---
    if "Slab and Beam Formwork" in created_tasks and "Slab and Beam Steel Reinforcement" in created_tasks:
        ifc_file.create_entity("IfcRelSequence", GlobalId=ifcopenshell.guid.new(), OwnerHistory=owner_history, RelatingProcess=created_tasks["Slab and Beam Formwork"], RelatedProcess=created_tasks["Slab and Beam Steel Reinforcement"], SequenceType="FINISH_START")
        print("Created example sequence relationship.")

    # --- 5. Save the Final Model ---
    ifc_file.write(output_ifc_path)
    print(f"\nSuccessfully created new 5D model from ontology at: {output_ifc_path}")


# --- Run the full process ---
target_guid = "2FUQJjpVj9wBkoUxY59XRk"
create_5d_model_from_ontology(target_guid)



Querying SPARQL endpoint...
Query successful.
Successfully processed SPARQL results into a structured format.

Found target element in IFC: Concrete-Rectangular Beam:PBEAM_230X300:367477
Created IfcTask: 'Slab and Beam Concrete Casting'
Created IfcTask: 'Slab and Beam Formwork'
Created IfcTask: 'Slab and Beam Steel Reinforcement'
Created IfcTask: 'Slab and Beam Curring'
Created example sequence relationship.

Successfully created new 5D model from ontology at: ../Models/Duplex_House_5D_From_Ontology.ifc


In [ ]:
def verify_5d_model(output_ifc_path, building_element_guid):

    print("\n" + "="*50 + "\n--- VERIFICATION ---")
    
    try:
        ifc_file = ifcopenshell.open(output_ifc_path)
        building_element = ifc_file.by_guid(building_element_guid)
        if not building_element:
            print(f"❌ VERIFICATION FAILED: Could not find building element with GUID {building_element_guid}")
            return
    except Exception as e:
        print(f"❌ VERIFICATION FAILED: Could not open or read file '{output_ifc_path}': {e}")
        return

    print(f"Verifying model for element: '{building_element.Name}'")

    # 1. Find all tasks assigned to the building element
    all_product_assignments = ifc_file.by_type("IfcRelAssignsToProduct")
    assigned_tasks = [obj for rel in all_product_assignments if rel.RelatingProduct == building_element for obj in rel.RelatedObjects if obj.is_a("IfcTask")]
    
    if not assigned_tasks:
        print("❌ No tasks found assigned to this element.")
        return

    print(f"\nFound {len(assigned_tasks)} tasks assigned to the element.")

    # 2. For each task, verify its data
    all_process_assignments = ifc_file.by_type("IfcRelAssignsToProcess")
    all_control_assignments = ifc_file.by_type("IfcRelAssignsToControl")

    for task in assigned_tasks:
        print(f"\n--- Verifying Task: '{task.Name}' ---")
        
        # Verify TaskTime
        if task.TaskTime:
            print("  ✅ TaskTime data found:")
            print(f"    - ScheduleStart: {task.TaskTime.ScheduleStart}")
            print(f"    - ActualStart:   {task.TaskTime.ActualStart}")
        else:
            print("  ❌ TaskTime data NOT found.")

        # Verify Resources
        resources_for_this_task = [obj for rel in all_process_assignments if rel.RelatingProcess == task for obj in rel.RelatedObjects]
        if resources_for_this_task:
            print(f"  ✅ Found {len(resources_for_this_task)} assigned resources:")
            for resource in resources_for_this_task:
                psets = ifcopenshell.util.element.get_psets(resource, psets_only=True)
                print(f"    - Resource: '{resource.Name}', Data: {psets.get('Pset_ResourceQuantities', 'No Pset')}")
        else:
            print("  ❌ No assigned resources found.")

        # Verify Costs
        costs_for_this_task = [obj for rel in all_control_assignments if rel.RelatingControl == task for obj in rel.RelatedObjects if obj.is_a("IfcCostItem")]
        if costs_for_this_task:
            print(f"  ✅ Found {len(costs_for_this_task)} assigned cost items:")
            for cost_item in costs_for_this_task:
                costs = {}
                if cost_item.CostValues:
                    for cost_value in cost_item.CostValues:
                        costs[cost_value.Name] = cost_value.AppliedValue.wrappedValue
                print(f"    - Cost Item: '{cost_item.Name}', Values: {costs}")
        else:
            print("  ❌ No assigned cost items found.")

In [10]:
verify_5d_model(
    output_ifc_path="../Models/Duplex_House_5D_From_Ontology.ifc",
    building_element_guid=target_guid
)



--- VERIFICATION ---
Verifying model for element: 'Concrete-Rectangular Beam:PBEAM_230X300:367477'

Found 4 tasks assigned to the element.

--- Verifying Task: 'Slab and Beam Concrete Casting' ---
  ✅ TaskTime data found:
    - ScheduleStart: 04-09-2023
    - ActualStart:   08-09-2023
  ✅ Found 1 assigned resources:
    - Resource: 'Concrete', Data: {'BudgetedUnits': 30.0, 'ActualUnits': 44.0, 'id': 22568}
  ✅ Found 3 assigned cost items:
    - Cost Item: 'Equipment Cost', Values: {'Budgeted': 26068.0, 'Actual': 24497.0}
    - Cost Item: 'Labor Cost', Values: {'Budgeted': 11180.0, 'Actual': 5229.0}
    - Cost Item: 'Material Cost', Values: {'Budgeted': 24996.0, 'Actual': 15533.0}

--- Verifying Task: 'Slab and Beam Formwork' ---
  ✅ TaskTime data found:
    - ScheduleStart: 27-08-2023
    - ActualStart:   29-08-2023
  ✅ Found 1 assigned resources:
    - Resource: 'FormWork', Data: {'BudgetedUnits': 50.0, 'ActualUnits': 62.0, 'id': 22589}
  ✅ Found 3 assigned cost items:
    - Cost Ite